In [1]:
import pandas as pd

labels = pd.read_csv('label_names.csv',header=None)
label_names = {}
for i in labels[0]:
    label_names[i.split(';')[0]] = i.split(';')[1].split("'")[1]
num_classes = len(label_names)
print(num_classes)

894


In [2]:
# Usando los 64 primeros label si no se le pone etiqueta else
label_use = []
for i in range(1,32):
    label_use.append(label_names[str(i)])
label_use.append('else')
num_classes = len(label_use)
print('numero de clases: ' + str(num_classes))
print('labels:')
print(label_use)

numero de clases: 32
labels:
['book', 'bottle', 'cabinet', 'ceiling', 'chair', 'cone', 'counter', 'dishwasher', 'faucet', 'fire\xa0extinguisher', 'floor', 'garbage\xa0bin', 'microwave', 'paper\xa0towel\xa0dispenser', 'paper', 'pot', 'refridgerator', 'stove\xa0burner', 'table', 'unknown', 'wall', 'bowl', 'magnet', 'sink', 'air\xa0vent', 'box', 'door\xa0knob', 'door', 'scissor', 'tape\xa0dispenser', 'telephone\xa0cord', 'else']


In [3]:
from PIL import Image
from pathlib import Path
from nyuv2 import *
from toolbox_nyu_depth_v2 import *
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
import cv2

DATASET_DIR = Path('dataset_nyuv2')
labeled = LabeledDataset(DATASET_DIR / 'nyu_depth_v2_labeled.mat')
# save_dir = 'segmented'
# color_dir = save_dir + '/color/'
# depth_dir = save_dir + '/depth/'
# label_dir = save_dir + '/label/'

x1 = []
x2 = []
y = []
x_rgb = []
x_dep = []
y_res = []

for i in range(1449): # valor maximo 1449
    color, depth, label = labeled[i]

    # x1.append(cv2.resize(img_to_array(color)/255, dsize=(128,96), interpolation=cv2.INTER_CUBIC))
    # x2.append(np.resize(cv2.resize(img_to_array(depth)/255, dsize=(128,96), interpolation=cv2.INTER_CUBIC),(96,128,1)))
    # y.append(np.resize(cv2.resize(img_to_array(label), dsize=(128,96), interpolation=cv2.INTER_CUBIC),(96,128,1)))
    x_rgb.append(img_to_array(color)/255) 
    x_dep.append(img_to_array(depth)/255)
    y_res.append(img_to_array(label))

x_rgb = np.array(x_rgb)
x_dep = np.array(x_dep)
y_res = np.array(y_res)

In [4]:
def maximize_image(imagen, num):
    img_aux = np.zeros([imagen.shape[0],imagen.shape[1]],dtype=np.uint8)
    for i in range(imagen.shape[0]):
        for j in range(imagen.shape[1]):
            if imagen[i][j] >= num:
                img_aux[i][j] = num
                    
#     plt.imshow(img_aux)
#     plt.show()
    return np.array(img_aux)

In [5]:
def binaryze_image(imagen, num):
    img_aux = np.zeros([imagen.shape[0],imagen.shape[1]],dtype=np.uint8)
    for i in range(imagen.shape[0]):
        for j in range(imagen.shape[1]):
            if num < 65:
                if imagen[i][j] == num:
                    img_aux[i][j] = 1
            else:
                if imagen[i][j] >= num:
                    img_aux[i][j] = 1
                    
#     plt.imshow(img_aux)
#     plt.show()
    return np.array(img_aux)

In [6]:
# Preparar el resultado en un array

y_res_1 = []

for im in range(len(y_res)):
    y_res_1.append(list())
    y_res_1[im].append(np.array(maximize_image(y_res[im],32)))
y_res_1 = np.array(y_res_1)
print(y_res_1[0].shape)

(1, 480, 640)


In [7]:
print(y_res_1.shape)

(1449, 1, 480, 640)


In [15]:
import tensorflow as tf

# Preparar el resultado en diferentes arrays
y_res_2 = []

for im in range(len(y_res)):
    y_res_2.append(list())
    y_res_2[im].append(np.array(tf.keras.utils.to_categorical(np.array(y_res[im]), num_classes=32, dtype='int32')))

y_res_2 = np.array(y_res_2)
print(y_res_2[0].shape)

IndexError: index 33 is out of bounds for axis 1 with size 32

In [9]:
np.save('dataset_nyuv2/y_prepared_32', y_res_2, allow_pickle=False, fix_imports=False)

In [10]:
np.save('dataset_nyuv2/y_32', y_res_1, allow_pickle=False, fix_imports=False)

In [11]:
np.save('dataset_nyuv2/x1', x_rgb, allow_pickle=False, fix_imports=False)

In [12]:
np.save('dataset_nyuv2/x2', x_dep, allow_pickle=False, fix_imports=False)

In [2]:
import zipfile
import time
import os

directorio = 'D:/Ikasketak/UNIR/08_TFM/TFM/Modelo/dataset_nyuv2'

for archivo in os.listdir(directorio):
    if os.path.isfile(os.path.join(directorio, archivo)):
        if archivo.split('.')[-1] == 'zip':
            with zipfile.ZipFile(os.path.join(directorio, archivo), 'r') as zip_ref:
                zip_ref.extractall(os.path.dirname(os.path.join(directorio, archivo)))

            time.sleep(1)
            os.remove(os.path.join(directorio, archivo))

In [15]:
os.remove(os.path.join(directorio, archivo))

In [ ]:
directorio = 'D:/Ikasketak/UNIR/08_TFM/TFM/Modelo/dataset_nyuv2'
# Mirar todos los archivos del directorio
for archivo in os.listdir(directorio):
    # Si es carpeta
    if os.path.isdir(os.path.join(directorio, archivo)):
        # Crear documento txt para meter todos los nombres de los archivos a utilizar
        txt=open(os.path.join(directorio, archivo)+".txt","w") 

        n_dir = os.path.join(directorio, archivo)

        # Si es
        for archivo in os.listdir(n_dir):
            if archivo.split('-')[0] != 'a':
                txt.write(os.path.join(n_dir, archivo)+"\n")
        txt.close()

        print("Sí es una carpeta")

Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo
Sí es un archivo


In [1]:
# verificar si es carpeta
import os
if os.path.isfile("archivo/o_ruta"):
    print("Sí es un archivo")
else:
    print("No es archivo, o no existe")
    
if os.path.isdir("ruta/a/carpeta"):
    print("Sí es una carpeta")
else:
    print("No es una carpeta o no existe")

No es archivo, o no existe
No es una carpeta o no existe
